This notebook combines the setup and image processing scripts for Omnipose, following the recommended installation procedures for Jupyter environments.

This notebook sets up the required dependencies for Omnipose using a conda virtual environment.

In [ ]:
# Create a new conda virtual environment named 'omnipose_env'
#!conda create --name omnipose_env python=3.9.13

# Activate the virtual environment
!conda activate omnipose_env

# Install the required dependencies
#%conda install -c conda-forge pytest pathlib scikit-image

# Note: 'omnipose' might need to be installed through pip or other means if it's not available in conda channels
#!pip install omnipose


The following section contains the Omnipose image processing code.

In [1]:
# Import dependencies
import numpy as np
from cellpose_omni import models, core

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? {}'.format(use_GPU))

# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
plt.style.use('dark_background')


ImportError: Numba needs NumPy 1.24 or less

Now that I have everything set up I need to load my images, which can be done in a variety of ways but here we will save them as variable "files" by importing a path and then matching file names with extensions and keywords



In [2]:
from pathlib import Path

basedir = r'C:\Users\MicrobeJ\Downloads\omnipose_multichannel\sequence'
files = [str(p) for p in Path(basedir).glob("Max.tif*")]

from cellpose_omni import io, transforms
from omnipose.utils import normalize99
imgs = [io.imread(f) for f in files]

# print some info about the images.
for i in imgs:
    print('Original image shape:',i.shape)
    print('data type:',i.dtype)
    print('data range: min {}, max {}\n'.format(i.min(),i.max()))
nimg = len(imgs)
print('\nnumber of images:',nimg)

fig = plt.figure(figsize=[40]*2,frameon=False) # initialize figure
print('\n')
for k in range(len(imgs)):
    img = transforms.move_min_dim(imgs[k]) # move the channel dimension last
    if len(img.shape)>2:
        # imgs[k] = img[:,:,1] # could pick out a specific channel
        imgs[k] = np.mean(img,axis=-1) # or just turn into grayscale 
        
    imgs[k] = normalize99(imgs[k])
    # imgs[k] = np.pad(imgs[k],10,'edge')
    print('new shape: ', imgs[k].shape)
    plt.subplot(1,len(files),k+1)
    plt.imshow(imgs[k],cmap='gray')
    plt.axis('off')

ImportError: Numba needs NumPy 1.24 or less

## Decide Which Model to use
Here we will list the models and choose which model to use. I am assuming we will almost always use the bact_phase_omni or bact_phase_cp models


In [4]:
from skimage.io import imread, imsave
import numpy as np
from pathlib import Path
from cellpose_omni import models, utils, io
import time

# Initialize model
model_name = 'bact_phase_omni'
model = models.CellposeModel(gpu=False, model_type=model_name)

# Parameters
chans = [0,0]  # Segment based on first channel, no second channel
n = range(len(files))  # Segment all images
mask_threshold = -1
flow_threshold = 0
niter = None
omni = True
cluster = True
resample = True
verbose = 0
affinity_seg = 0

# Run segmentation
tic = time.time()
masks, flows, styles = model.eval(
    [imread(f)[..., 0] for f in files],  # Reading only the first channel for segmentation
    channels=chans,
    mask_threshold=mask_threshold,
    flow_threshold=flow_threshold,
    niter=niter,
    omni=omni,
    cluster=cluster,
    resample=resample,
    verbose=verbose,
    affinity_seg=affinity_seg
)
net_time = time.time() - tic
print(f'total segmentation time: {net_time}s')

# Apply masks and save images
for idx, mask in enumerate(masks):
    original_image = imread(files[idx])  # Read the original multi-channel image
    if original_image.shape[:2] != mask.shape:
        print(f"Dimension mismatch for {files[idx]}")
        continue

    # Extend mask dimensions to match the multi-channel image
    mask_multi_channel = np.repeat(mask[:, :, np.newaxis], original_image.shape[2], axis=2)

    # Apply the mask to all channels
    masked_image = original_image * (mask_multi_channel > 0)

    # Save masked image and other outputs
    filename = Path(files[idx]).name
    base_name = Path(filename).stem
    imsave(f"{basedir}/{base_name}_masked.tif", masked_image.astype(np.uint16))

    outlines = utils.outlines_list(mask)
    io.outlines_to_text(f"{basedir}/{base_name}_outlines.txt", outlines)
    imsave(f"{basedir}/{base_name}_mask.tif", mask.astype(np.uint16))


ImportError: Numba needs NumPy 1.24 or less

In [3]:
from cellpose_omni import plot
import omnipose

for idx,i in enumerate(n):

    maski = masks[idx] # get masks
    bdi = flows[idx][-1] # get boundaries
    flowi = flows[idx][0] # get RGB flows 

    # set up the output figure to better match the resolution of the images 
    f = 10
    szX = maski.shape[-1]/mpl.rcParams['figure.dpi']*f
    szY = maski.shape[-2]/mpl.rcParams['figure.dpi']*f
    fig = plt.figure(figsize=(szY,szX*4))
    fig.patch.set_facecolor([0]*4)
    
    plot.show_segmentation(fig, omnipose.utils.normalize99(imgs[i]), 
                           maski, flowi, bdi, channels=chans, omni=True)

    plt.tight_layout()
    plt.show()


base = os.path.splitext(image_name)[0]
outlines = utils.outlines_list(masks)
io.outlines_to_text(base, outlines)
io.save_to_png(images, masks, flows, image_names)   

ImportError: Numba needs NumPy 1.24 or less

In [ ]:
from cellpose_omni import io, plot

# image_name is file name of image
# masks is numpy array of masks for image
base = os.path.splitext(image_name)[0]
outlines = utils.outlines_list(masks)
io.outlines_to_text(base, outlines)